<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Prelude" data-toc-modified-id="Prelude-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Prelude</a></span></li><li><span><a href="#Utilities-(as-before)" data-toc-modified-id="Utilities-(as-before)-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Utilities (as before)</a></span></li><li><span><a href="#Finding-Shortest-Path-(as-before)" data-toc-modified-id="Finding-Shortest-Path-(as-before)-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Finding Shortest Path (as before)</a></span></li><li><span><a href="#Finding-Shortest-Delivery-Route-(as-before)" data-toc-modified-id="Finding-Shortest-Delivery-Route-(as-before)-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Finding Shortest Delivery Route (as before)</a></span></li><li><span><a href="#Time-Handling" data-toc-modified-id="Time-Handling-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Time Handling</a></span></li><li><span><a href="#Class-Recorder" data-toc-modified-id="Class-Recorder-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Class Recorder</a></span></li><li><span><a href="#Class-Parcel" data-toc-modified-id="Class-Parcel-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Class Parcel</a></span></li><li><span><a href="#Class-Customer" data-toc-modified-id="Class-Customer-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Class Customer</a></span></li><li><span><a href="#Class-Driver" data-toc-modified-id="Class-Driver-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Class Driver</a></span></li><li><span><a href="#Class-Delivery-Centre" data-toc-modified-id="Class-Delivery-Centre-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Class Delivery Centre</a></span></li><li><span><a href="#Simulation" data-toc-modified-id="Simulation-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Simulation</a></span><ul class="toc-item"><li><span><a href="#Parameters-from-Specification" data-toc-modified-id="Parameters-from-Specification-11.1"><span class="toc-item-num">11.1&nbsp;&nbsp;</span>Parameters from Specification</a></span></li><li><span><a href="#Generate-Input-Data" data-toc-modified-id="Generate-Input-Data-11.2"><span class="toc-item-num">11.2&nbsp;&nbsp;</span>Generate Input Data</a></span></li><li><span><a href="#Simulation-Routine" data-toc-modified-id="Simulation-Routine-11.3"><span class="toc-item-num">11.3&nbsp;&nbsp;</span>Simulation Routine</a></span></li><li><span><a href="#Model-Verification" data-toc-modified-id="Model-Verification-11.4"><span class="toc-item-num">11.4&nbsp;&nbsp;</span>Model Verification</a></span></li></ul></li></ul></div>

# Prelude

In [1]:
import matplotlib.pyplot as plt
import pulp
import math
import random
import pandas as pd
import numpy as np
import simpy

# Utilities (as before)

## Points and Distances

In [2]:
def dist(p1, p2):
    (x1, y1) = p1
    (x2, y2) = p2
    return int(math.sqrt((x1-x2)**2+(y1-y2)**2))

## PlotMap

In [3]:
def plotMap(G, T=[], P=[], W=None,
            style='r-o', lw=1, ms=3, 
            styleT='go', msT=5,
            styleP='b-o', lwP=3, msP=1,
            stylePT='go', msPT=7,
            styleW='bo', msW=9,
            text=None, grid=False):
    fig = plt.gcf()
    fig.set_size_inches(6, 6)
    V, E = G

    if not grid:
        plt.axis('off')
    plt.plot( [ p[0] for p in V ], [ p[1] for p in V ], 'ro', lw=lw, ms=ms)
    for (p, q) in E:
        plt.plot( [ p[0], q[0] ], [ p[1], q[1] ], 'r-o', lw=lw, ms=ms)
    for t in T:
        plt.plot( [ t[0] ], [ t[1] ], 
                  styleT, ms=msT)
    plt.plot( [ p[0] for p in P ],
              [ p[1] for p in P ], 
              styleP, lw=lwP, ms=msP)
    for p in P:
        if p in T:
            plt.plot( [ p[0] ], [ p[1] ], 
                      stylePT, ms=msPT)
    if W is not None:
        plt.plot( [ W[0] ], [ W[1] ], 
                      styleW, ms=msW)
    if text is not None:
        maxX = max([p[0] for p in V])
        plt.text(0.8*maxX, 0, text)
    if grid:
        plt.grid()
    plt.show()

## Add Targets

In [4]:
def addTargets(M, T):
    V, E = M
    E = E.copy()
    V = V.copy()
    for t in T:
        minD = math.inf
        minE = None
        for e in E:
            P, Q = e
            distT = dist(P, t)+dist(t, Q)-dist(P, Q)
            if distT < minD:
                minD = distT
                minE = e
        P, Q = minE
        E.remove( (P, Q) )
        E.append( (P, t) )
        E.append( (t, Q) )
        V.append(t)
    return V, E

## Generate Warehouse Location

In [5]:
def generateWarehouseLocation(M):
    V, _ = M
    W = random.sample(V, k=1)[0]
    return W

## Time Handling

**Convention:** In this project we measure time in seconds. The simulation will start at 0:00. Time related methods will be added as they are needed.

`timestamp(t)` generates a timestamp string in the form `[dd] hh:mm:ss.d`

In [6]:
    def timestamp(t):
        day = int(t)//(24*3600)
        t = t - day*24*3600
        hour = int(t)//3600
        t = t - hour*3600
        mins = int(t)//60
        t = t - mins*60
        secs = int(t)
        t = int(round((t - secs)*10))
        return f"[{day:2d}] {hour:02d}:{mins:02d}:{secs:02d}.{t:1d}"

In [7]:
timestamp(24*3600*3+17*3600+615.1)

'[ 3] 17:10:15.1'

In [8]:
def nextHour(env, hour):
    beginningOfDay = int(env.now//(24*3600))*24*3600
    timeOfDay = env.now-beginningOfDay
    if hour*3600 > timeOfDay:
        return hour*3600 - timeOfDay
    else:
        return hour*3600 + 24*3600 - timeOfDay

In [9]:
def day(now):
    return int(now//(24*3600))

# Finding Shortest Path (as before)

In [10]:
def dist(p1, p2):
    (x1, y1) = p1
    (x2, y2) = p2
    return int(math.sqrt((x1-x2)**2+(y1-y2)**2))

In [11]:
def pathLength(P):
    return 0 if len(P)<=1 else \
            dist(P[0], P[1])+pathLength(P[1:])

In [12]:
def shortestPath(M, A, B):

    def h(p): 
        return pathLength(p)+dist(p[-1],B)
    
    # candidates C are pairs of the path so far and 
    # the heuristic function of that path, 
    # sorted by the heuristic function, as maintained by
    # insert function
    def insert(C, p):
        hp = h(p)
        c = (p, hp)
        for i in range(len(C)):
            if C[i][1]>hp:
                return C[:i]+[c]+C[i:]
        return C+[c]   
        
    V, E = M
    assert(A in V and B in V)    
    C = insert([], [A])

    while len(C)>0:
        # take the first candidate out of the list of candidates
        path, _ = C[0]
        C = C[1:]
        if path[-1]==B:
            return path
        else:
            for (x, y) in E:
                if path[-1]==x and y not in path:
                    C = insert(C, path+[y])
                elif path[-1]==y and x not in path:
                    C = insert(C, path+[x])
    return None

# Finding Shortest Delivery Route (as before)

## Iterative Integer Programming

In [13]:
def createTables(M, T):

    def reverse(P):
        return [ P[-i] for i in range(1,len(P)+1) ]
    
    def index(x, L):
        for i in range(len(L)):
            if x==L[i]: 
                return i
        return None

    n = len(T)
    d = [ [ math.inf for t in T ] for t in T ]
    p = [ [ None for t in T ] for t in T ]
    for i in range(n):
        d[i][i] = 0
        p[i][i] = [ T[i] ]
    for i in range(n):
        for j in range(n):
            if p[i][j] is None:
                s = shortestPath(M, T[i], T[j])
                d[i][j] = d[j][i] = pathLength(s)
                p[i][j] = s
                p[j][i] = reverse(s)
                for m in range(len(s)-1):
                    smi = index(s[m], T)
                    if smi is None:
                        continue
                    for l in range(m+1, len(s)):
                        sli = index(s[l], T)
                        if sli is None:
                            continue  
                        sub = s[m:l+1]
                        if p[smi][sli] is None:
                            p[smi][sli] = sub
                            p[sli][smi] = reverse(sub)
                            d[smi][sli] = d[sli][smi] = pathLength(sub)
    return d,p               

In [14]:
def roundtrips(x, n):
    
    def isElem(x, l):
        for i in range(len(l)):
            if l[i]==x:
                return True
        return False

    def startpoint(trips):
        for i in range(n):
            for t in trips:
                if isElem(i, t):
                    break
            else:
                return i
    
    def totalLength(trips):
        s=0
        for i in range(0, len(trips)):
            s += len(trips[i])-1
        return s

    trips = []
    while totalLength(trips)<n:
        start = startpoint(trips)
        trip = [ start ]
        i = start
        while len(trip) < n-totalLength(trips):
            for j in range(0, n):
                if pulp.value(x[i][j])==1:
                    trip.append(j)
                    i=j
                    break        
            if pulp.value(x[trip[-1]][start])==1:
                trip.append(start)
                break
        trips.append(trip)
    return sorted(trips, key=lambda t: len(t), reverse=True)

In [15]:
import time

def createLoop(M, T, timing=False):
    
    if timing:
        start_time = time.time()
        last_time = time.time()
        
    D, P = createTables(M, T)   # These are the distances between customers and warehouse only
    
    if timing:
        print(f"createTables:   {time.time()-start_time:6.2f}s")
        last_time = time.time()
        
    n = len(T)

    if n==1:
        return T
        
    # create variables
    x = pulp.LpVariable.dicts("x", ( range(n), range(n) ),
                            lowBound=0, upBound=1, cat=pulp.LpInteger)
    # create problem
    prob = pulp.LpProblem("Loop",pulp.LpMinimize)
    # add objective function
    prob += pulp.lpSum([ D[i][j]*x[i][j] 
                             for i in range(n) for j in range(n) ])
    # add constraints
    constraints=0
    for j in range(n):
        prob += pulp.lpSum([ x[i][j] for i in range(n) if i!=j ]) ==1
    constraints += n
    for i in range(n):
        prob += pulp.lpSum([ x[i][j] for j in range(n) if i!=j ]) ==1
    constraints += n
    for i in range(n):
        for j in range(n):
            if i!=j:
                prob += x[i][j]+x[j][i] <= 1
                constraints += 1
    # initialise solver
    solvers = pulp.listSolvers(onlyAvailable=True)
    solver = pulp.getSolver(solvers[0], msg=0)
    prob.solve(solver)

    if timing:
        print(f"Solver:         {time.time()-last_time:6.2f}s {constraints:6,d} Constraints")
        last_time = time.time()

    trips = roundtrips(x, n)
    while len(trips)>1:
        longest = max([ len(t) for t in trips ])
        for t in trips:
            if len(t)<longest:
                prob += pulp.lpSum([ x[t[i]][t[i+1]] + x[t[i+1]][t[i]]
                                        for i in range(0,len(t)-1) ]) <= len(t)-2
                constraints += 1
            else:
                longest = math.inf
        prob.solve(solver)
        
        if timing:
            print(f"Solver:         {time.time()-last_time:6.2f}s {constraints:6,d} Constraints")
            last_time = time.time()

        trips = roundtrips(x, n)
        
    trip = trips[0]
    loop = []
    for k in range(len(trip)-1):
        sub = P[trip[k]][trip[k+1]]
        loop += sub if len(loop)==0 else sub[1:]
        
    if timing:
        print(f"createLoop:     {time.time()-start_time:6.2f}s")

    return loop

## Heuristic Algorithm

In [16]:
def FW(M):
    
    V, E = M

    n = len(V)
    d = [ [ math.inf for j in range(n) ] for i in range(n) ]
    p = [ [ None for j in range(n) ] for i in range(n) ]

    for (A, B) in E:
        a = V.index(A)
        b = V.index(B)
        d[a][b] = d[b][a] = dist(A, B)
        p[a][b] = [A, B]
        p[b][a] = [B, A]
    
    for i in range(n):
        d[i][i] = 0
        p[i][i] = [V[i]]
    
    for k in range(n):
        for i in range(n):
            for j in range(n):
                dk = d[i][k] + d[k][j]
                if d[i][j] > dk:
                    d[i][j] = dk
                    p[i][j] = p[i][k][:-1] + p[k][j]
                    
    return d, p

In [17]:
def createLoopH(M, T, timing=False):
    
    def makeLoop(L):
        loop = []
        for i in range(len(L)-1):
            A = L[i]
            B = L[i+1]
            a = V.index(A)
            b = V.index(B)
            sub = P[a][b]
            loop += sub if len(loop)==0 else sub[1:]
        return loop
        
    if timing:
        start_time = time.time()
        last_time = time.time()
        
    V, E = M
    D, P = FW(M)   # note these are the distances between all vertices in M (and T)
    
    if timing:
        print(f"createTables:   {time.time()-start_time:6.2f}s")
        last_time = time.time()
        
    W = T[0]
    customers = T[1:]
    if len(T)==1:
        L = T
    elif len(T)<=3:
        L = T + [T[0]]
    else:
        L = T[:3]+[T[0]]
        T = T[3:]
        while len(T)>0:
            minExt = math.inf
            minInd = None
            selInd = None
            for k in range(len(T)):
                C = T[k]
                c = V.index(C)
                for i in range(0, len(L)-1):
                    A = L[i]
                    B = L[i+1]
                    a = V.index(A)
                    b = V.index(B)
                    ext = D[a][c] + D[c][b] - D[a][b]
                    if ext<minExt:
                        minExt, minInd, selInd = ext, i+1, k
            L = L[:minInd]+[T[selInd]]+L[minInd:]
            T = T[:selInd]+T[selInd+1:]
            
    if timing:
        print(f"createLoopH:    {time.time()-start_time:6.2f}s")

    return makeLoop(L)

# Class Recorder

We willl use a class Recorder as a reference point for capturing data during the simulation. There will be only one recorder. It will be created at the beginning of every simulation run. Every entity will carry a reference to the Recorder. 

In [18]:
import time

class Recorder:

    def __init__(self, env, M, W, C, days, operational_cost,
                 log=False, plot=False, timing=False):
        self.env = env
        self.M = M
        self.W = W
        self.C = C
        self.days = days
        self.total_operational_cost = 0  # Track total operational cost
        self.operational_cost = operational_cost
        self.log = log
        self.plot = plot
        self.timing = timing
        self.start_time = time.time()
        self.last_time = self.start_time
        self.cum_timer = {}
        self.daily_tour_length = [0] * days
        self.daily_working_time = [0] * days
        self.daily_cost = [0] * days
        self.current_day = 0
        self.hourly_rate = hourly_rate
        self.daily_payment = [0] * days
        self.total_payment = 0  

            
        Customer.REGISTER = []
        Parcel.REGISTER = []
    
    def timer(self, s):
        t = time.time()
        Δt = t-self.last_time
        if self.timing:
            print(f"==== t: {t-self.start_time:6.2f}s "
                  f"Δt: {Δt:6.2f}s [{s:s}]")
        if s in self.cum_timer:
            self.cum_timer[s] += Δt
        else:
            self.cum_timer[s] = Δt
        self.last_time = t

    def reportTimer(self):
        print()
        for k in sorted(self.cum_timer, key=lambda x: self.cum_timer[x], reverse=True):
            print(f"==== Σ: {self.cum_timer[k]:6.2f}s [{k:s}]")
        print(f"==== -----------------------")
        print(f"==== Σ: {time.time()-self.start_time:6.2f}s Total")

 
    def trace(self, event):
        if self.log:
            print(timestamp(self.env.now), event)

    # def recordDailyTourLength(self, tour_length):
    #     self.daily_tour_length[self.current_day] += D.self.tour_length

    # def recordDailyWorkingTime(self, worked_time):
    #     self.daily_working_time[self.current_day] += worked_time

    # def calculateDailyCosts(self):
    #     # Calculate the daily cost based on tour length and working time
    #     daily_cost = self.daily_tour_length[self.current_day] * self.operational_cost
    #     self.daily_cost[self.current_day] = daily_cost
    #     self.total_operational_cost += daily_cost 
    #     print(f"Tour length for the day {self.daily_tour_length[self.current_day]:.2f}m")
    #     print(f"Operational for the day {self.daily_cost[self.current_day]:.2f}€")
    #     print(f"Total operational cost {self.total_operational_cost}€")
       
        

    # def recordDailyPayment(self):
    #     daily_payment = max(self.daily_working_time[self.current_day] * self.hourly_rate, minimum_pay)
    #     self.daily_payment[self.current_day] = daily_payment
    #     self.total_payment += daily_payment
    #     print(f"Driver's Payment for the day {self.daily_payment[self.current_day]:.2f}€")
    #     print(f"Driver's total Payment {self.total_payment}€")

    # def nextDay(self):
    #     self.current_day += 1

    def finish(self):
    # Calculate daily costs for all recorded days
        # for day in range(self.days):
        #     self.calculateDailyCosts()
        
        # # Calculate daily payments for all recorded days
        # for day in range(self.days):
        #     self.recordDailyPayment()
        #     self.nextDay()
            self.env = None


    

# Class Parcel

<img src='img/EventGraph Parcel.png' width=500/>

Parcels follow through a sequence of states:
- processing
- in transit (from manufacture to distribution centre)
- arrived in distribution centre
- ready for delivery 
- out for delivery
- customer not present
- returned to distribution centre
- delivered

In [19]:
class Parcel:
    
    REGISTER = []
    
    def __init__(self, rec, i, cust, custIndex):
        self.rec = rec
        self.i = i  # row index in data frames of input data
        self.dest = cust.location
        self.custIndex = custIndex
        self.status = [ 'processing' ] # status record and
        self.timing = [ self.rec.env.now ]      # timing
        assert(len(Parcel.REGISTER)==i)
        Parcel.REGISTER += [ self ] 
    
    # factory method ensures that there is only
    # one Parcel per location
    def getParcel(rec, i, location, custIndex):
        for p in Parcel.REGISTER:
            if p.i == i:
                return p
        return Parcel(rec, i, location, custIndex)

    
    def __str__(self):
        return f"Parcel: {self.i:3d} ({self.custIndex:3d})"

    def index(self):
        return self.i
    
    def destination(self):
        return self.dest
    
    def __reg(self, state):
        self.status += [ state ]
        self.timing += [ self.rec.env.now ]
        self.rec.trace(str(self)+" "+state)
        
    def arrivedAtDeliveryCentre(self):
        self.__reg('arr at delivery centre')
        
    def outForDelivery(self): 
        self.__reg('out for delivery')
        
    def returnFromDelivery(self):
        self.__reg('return from delivery')
    

# Class Customer

<img src='img/EventGraph Customer.png' width=500/>

In [20]:
class Customer:
    
    REGISTER = []

    def __init__(self, rec, location):
        self.rec = rec
        self.location = location
        self.i = len(Customer.REGISTER)
        Customer.REGISTER += [ self ]
        self.atHome = True
        self.answersDoor = False
        self.parcelsReceived = []
        rec.env.process(self.process())
        
    def __str__(self):
        return f"Customer: {self.i:2d} {str(self.location):s}"
    
    # factory method ensures that there is only
    # one customer per location
    def getCustomer(rec, location):
        for c in Customer.REGISTER:
            if c.location == location:
                return c
        return Customer(rec, location)
    
    def leaveHouse(self):
        assert(self.atHome and not self.answersDoor)
        # self.rec.trace(str(self)+" leaves house")
        self.atHome = False
     
    def returnHome(self):
        assert(not self.atHome)
        # self.rec.trace(str(self)+" returns home")
        self.atHome = True
        
    def answerDoor(self):
        if self.atHome:
            yield self.rec.env.timeout(random.expovariate(1/AVERAGE_TIME_ANSWER_DOOR))
            self.rec.trace(str(self) + " answers door")
            self.answersDoor = True
           # Simulate additional handover time for the parcel
            handover_time = random.expovariate(1 / mean_handover_time_per_parcel)  # Mean time of 10 seconds
            yield self.rec.env.timeout(handover_time)
            self.rec.trace(str(self)+" handover")
        else:
            yield self.rec.env.timeout(WAIT_TIME_IF_CUSTOMER_DOESNT_ANSWER_DOOR)
            self.rec.trace(str(self) + " not at home")

           
    def acceptParcel(self, parcel):
        assert(self.answersDoor)
        self.parcelsReceived += [parcel]
        self.rec.trace(str(self)+" accepts "+str(parcel))
        
    def signOff(self):
        assert(self.answersDoor)
        self.rec.trace(str(self)+" signs off")
        self.answersDoor = False
        
    def process(self):
        yield self.rec.env.timeout(nextHour(self.rec.env, 8))
        while day(self.rec.env.now)<self.rec.days:
            # in a refinement we may use random times
            self.leaveHouse()
            yield self.rec.env.timeout(nextHour(self.rec.env, 18))
            self.returnHome()
            yield self.rec.env.timeout(nextHour(self.rec.env, 8))

# Class Driver

<img src='img/EventGraph Driver.png' width=500/>

In [21]:
class Driver:
    
    def __init__(self, rec, DC):
        self.rec = rec
        self.DC = DC
        self.location = None
        self.parcels = None
        self.tour = None
        self.total_tour_length = 0
        self.tour_length = 0  # Initialize tour length
        self.rec.env.process(self.process())
        self.arrival_time = 0  # Record arrival time
        self.departure_time = 0  # Record departure time
        self.total= 0
        self.daily = 0
        self.worked_time = 0  # Initialize worked time
        self.operational_cost = operational_cost
        self.total_operational_cost = 0
        self.daily_operational_cost = 0

        
    # activity
    def __drive(self, target):
        assert(self.tour[0] == self.location)
        self.tour_length = sum(dist(self.tour[i], self.tour[i + 1]) for i in range(len(self.tour) - 1))  # Calculate tour length
        while self.location!=target:
            d = dist(self.location, self.tour[1])
            yield self.rec.env.timeout(d / AVERAGE_SPEED)
            self.location = self.tour[1]
            self.tour = self.tour[1:]
            
            
        assert(self.tour[0] == self.location == target)
    
    def arriveForWork(self):
        self.arrival_time = self.rec.env.now 
        self.location = self.DC.W
        self.parcels = []
        self.returns = []
        self.tour = [ self.DC.W ]
        self.rec.trace("Driver arrives for work")
        
    def leaveForDelivery(self, tour, parcels):
        self.tour, self.parcels = tour, parcels
        self.rec.trace(f"Driver leaves for delivery " \
                       f"of {len(parcels):d} parcels")
        
    def process(self):
        yield self.rec.env.timeout(nextHour(self.rec.env, 18))
        while day(self.rec.env.now)<self.rec.days:
            self.arriveForWork()
            tour, parcels = self.DC.sendForDelivery()
            yield self.rec.env.timeout(PREP_TIME_PER_PARCEL*len(parcels))
            self.leaveForDelivery(tour, parcels) 
            
            
            while len(self.parcels)>0:
                
                # drive to customer
                custLocation = self.parcels[0].dest
                cust = Customer.getCustomer(self.rec, custLocation)
                self.rec.trace("Driver drives to "+str(cust))
                yield from self.__drive(custLocation)
                self.rec.trace("Driver arrived at "+str(cust))
                # call at customer
                yield from cust.answerDoor()
                
                if cust.answersDoor:
                    while len(self.parcels)>0 and \
                            custLocation == self.parcels[0].dest:
                        cust.acceptParcel(self.parcels[0])
                        yield self.rec.env.timeout(random.expovariate(1/10))
                        self.parcels = self.parcels[1:]
                    cust.signOff()
                    yield self.rec.env.timeout(random.expovariate(1/10))
                else:
                    while len(self.parcels)>0 and  \
                            custLocation == self.parcels[0].dest:
                        self.returns += self.parcels[0]
                        self.parcels = self.parcels[1:]

            # return to delivery centre
            self.rec.trace("Driver returns to delivery centre")
            yield from self.__drive(self.DC.W)
            self.rec.trace("Driver arrived at delivery centre")
            
            for parcel in self.returns:
                self.DC.returnFromDelivery(parcel)
                yield self.rec.env.timeout(RETURN_TIME_PER_PARCEL)
            yield self.rec.env.timeout(600)
            self.dayEndProcedure()
            self.rec.trace("Driver goes home")
            self.departure_time = self.rec.env.now  # Record departure time
            self.worked_time = self.departure_time - self.arrival_time
            # self.rec.recordDailyWorkingTime(self.worked_time)
            print(f"Driver's Working Time for the day: {self.worked_time:.2f} seconds")
        
            self.calculate_daily_payment()
            self.calculate_operational_cost()
            # self.rec.calculateDailyCosts()
            # Calculate and record daily tour length
            # self.rec.recordDailyTourLength(self.tour_length)
            # self.rec.recordDailyPayment()
            yield self.rec.env.timeout(nextHour(self.rec.env, 18))
            

    def dayEndProcedure(self):
        # Report and set up the cargo-bike for charging
        self.rec.trace("Day-end procedure: Reporting and setting up the cargo-bike for charging")
        yield self.rec.env.timeout(cargo_bike_charging)  # 10 minutes for the day-end procedure
        # self.rec.calculateDailyCosts()


    def calculate_daily_payment(self):
        # Calculate the daily payment based on the hours worked
        self.daily = max(hourly_rate * self.worked_time, minimum_pay)  # Minimum payment of 60€
        self.total += self.daily
        print(f"Driver's Payment for the day {self.daily:.2f}€")

    def calculate_operational_cost(self):
        self.total_tour_length += self.tour_length
        daily_operational_cost = self.tour_length * operational_cost
        self.total_operational_cost += daily_operational_cost  # Update total operational cost
        print(f"Distance traveled for the day: {self.tour_length} m")
        print(f"Operational cost for the day: {daily_operational_cost}€ ")
    
            


    
    


# Class Delivery Centre

<img src="img/EventGraph Delivery Centre.png" width=300/>

In [22]:
class DeliveryCentre:
    
    def __init__(self, rec, M, W, operational_cost):
        self.rec = rec
        self.M = M
        self.W = W
        self.limit = max_range
        self.operational_cost = operational_cost
        
        self.leftOver = []    # list of parcels
        self.parcels = []     # list of parcels scheduled for delivery
        self.dest = []        # list of unique customer destinations
        self.tour = None      # tour planned for delivery
    
    def __accept(self, parcel):
        custLoc = parcel.dest
        if custLoc not in self.dest:
            MT = addTargets(self.M, self.dest + [custLoc])
            self.rec.timer("addTarget")
            SH = createLoopH(MT, [self.W] + self.dest + [custLoc],  timing=self.rec.timing)
            self.rec.timer("createLoopH")
            if self.tour is None and pathLength(SH)<self.limit:
                self.parcels.append(parcel)
                self.dest += [custLoc]
            else: 
                S = createLoop(MT, [self.W] + self.dest + [custLoc], 
                               timing=self.rec.timing)
                self.rec.timer("createLoop")
                if pathLength(S)<self.limit:
                    self.parcels.append(parcel)
                    self.dest += [custLoc]
                    self.tour = S
                else:
                    self.leftOver.append(parcel)
        else:
            self.parcels.append(parcel)
        
    def acceptParcel(self, parcel):
        parcel.arrivedAtDeliveryCentre()
        self.__accept(parcel)
            
    def sendForDelivery(self):
        parcels = []
        if self.tour is None:
            MT = addTargets(self.M, self.dest)
            self.rec.timer("addTarget")
            self.tour = createLoop(MT, [self.W] + self.dest, 
                                   timing=self.rec.timing)
            self.rec.timer("createLoop")
        tour = self.tour
        addresses = self.dest
        
        # pick parcels in sequence to be delivered
        for i in range(1, len(tour)-1):
            dest = tour[i]
            for p in self.parcels:
                if p.dest == dest and p not in parcels:
                    parcels += [p]
                    p.outForDelivery()
         
        # arrange the left overs
        L = self.leftOver
        self.tour = None
        self.parcels = []
        self.leftOver = []
        self.dest = []
        for p in L:
            self.__accept(p)
        
        if self.rec.plot:
            plotMap(self.rec.M, T=addresses, P=tour, W=tour[0], 
                    text=f"Day {day(self.rec.env.now):2d}, {pathLength(tour):,d}m")

        return tour, parcels
                 
    def returnFromDelivery(self, parcel):
        parcel.returnFromDelivery()
        self.__accept(parcel)

    def getInventory(self):
        return len(self.parcels)+len(self.leftOver)




# Simulation

## Parameters from Specification

The time required for driving is based on the distance between way points at an average speed of 15km/h.

In [23]:
AVERAGE_SPEED = 15/3.6

The **cumulative preparation time** (route planning and 
sorting of the parcels in the delivery order and 
packing the cargo-bike) is assumed to be 50 sec per 
parcel to be delivered.

In [24]:
PREP_TIME_PER_PARCEL = 50

**Additional assumption:** The time to **process returned parcels** in the delivery centre is 30 sec per parce. 

In [25]:
RETURN_TIME_PER_PARCEL = 30

The average time to answer the door.

In [26]:
AVERAGE_TIME_ANSWER_DOOR = 40

In [27]:
mean_handover_time_per_parcel = 10  

In [28]:
WAIT_TIME_IF_CUSTOMER_DOESNT_ANSWER_DOOR = 60

In [29]:
max_range = 40000

In [30]:
cargo_bike_charging = 10*60

In [31]:
operational_cost = 8*0.01/1000

In [32]:
minimum_pay = 60

In [33]:
hourly_rate = 30/3600

## Generate Input Data

In [34]:
def generateDeliveryData(p, C, days, seed=0):
    ## p is the average number of parcels per day per customer
    ## C is the number of customers to be served
    ## days is the number of days for which data are to be generated.
    np.random.seed(seed)
    R = np.random.poisson(lam=len(C)*p, size=days)
    D = [ sorted(list(np.random.choice(range(len(C)), size=i))) for i in R ]
    return D

In [35]:
def generateInputData(D, log=False):

    R = [ len(d) for d in D ]
    N = sum(R)
    
    DAY_LENGTH = 24*3600    # measured in minutes
    DAY_START = 8*3600      # first delivery in the morning
    DAY_END = 17*3600     # last delivery during day time

    x = pd.DataFrame()

    x['iarr'] = [None]*N
    x['time'] = [None]*N
    x['day'] = [None]*N
    x['dest'] = [None]*N

    current_day = 0
    last_time = 0
    i = 0

    for d in D: # for each day

        if log:
            print("generating for day: ",current_day, D[current_day])
            
        time = current_day*DAY_LENGTH + DAY_START 

        for c in d: # for each customer that should get a 

            IARR = (DAY_END-DAY_START-2*3600) / len(d)  # estimated average IAT for the current day

            iat = random.expovariate(1.0/IARR)
            new_time = time + iat

            x.at[i, 'iarr'] = round(new_time - last_time,1)
            x.at[i, 'time'] = round(new_time - current_day*DAY_LENGTH , 1)
            x.at[i, 'day'] = current_day
            x.at[i, 'dest'] = c
            
            i += 1
            last_time = time = new_time
    
        current_day += 1
    
    return x

## Simulation Routine

In [36]:
def simulation(M, W, C, operational_cost, p=0.15, days=10, seed=0, 
               log=False, plot=False, timing=False):

    random.seed(seed)
    D = generateDeliveryData(p, C, days, seed)
    X = generateInputData(D, log=log)
    
    env = simpy.Environment()
    rec = Recorder(env, M, W, C, days,operational_cost,
                   log=log, plot=plot, timing=timing)

    print(f"Simulating delivery of {len(X):d} parcels "
          f"over {len(D):d} days to {len(C):d} customers")

    for c in C:
        Customer.getCustomer(rec, c)           
    DC = DeliveryCentre(rec, M, W, operational_cost)
    D = Driver(rec, DC)


    def generatorProcess(env):
        # generate the parcels based on input data x
        for i in range(len(X)):
            yield env.timeout(X.at[i, 'iarr'])
            custIndex = X.at[i, 'dest']
            custLoc = C[custIndex]
            cust = Customer.getCustomer(rec, custLoc)
            p = Parcel.getParcel(rec, i, cust, custIndex)
            DC.acceptParcel(p)
            # Perform day-end procedure after accepting each parcel
            D.dayEndProcedure()
         
    env.process(generatorProcess(env))
    env.run()

    rec.finish()
            
    total_payment = D.total # Initialize total payment variable
    # total_operational_cost = sum(rec.daily_cost)  # Calculate total operational cost
    total_operational_cost = D.total_operational_cost # Calculate total operational cost
    total_tour_length = D.total_tour_length  # Calculate total tour length
  
    
    print(f"Total Driver's Payment for {days} days: {total_payment:.2f}€")
    print(f"Total distance traveled: {total_tour_length} m")
    print(f"Total Operational Cost for {days} days: {total_operational_cost:.2f}€")


    if log:
        print(f"Delivery Centre Inventory: {DC.getInventory():d} parcels")

    return rec,D

## Model Verification

In [37]:
import pickle
with open('BolormaaData.pickled', 'rb') as f:
    M, C = pickle.load(f)

### 4 days of verification

In [38]:
random.seed(6725)
W = generateWarehouseLocation(M)
rec, driver= simulation(M, W, C, operational_cost, p=0.15, days=4, log=True)
rec.reportTimer()


generating for day:  0 [9, 9, 25, 29, 32, 39, 58, 72, 77, 79, 81, 82, 87, 88, 88, 99, 115, 127, 140, 142, 147, 147, 148]
generating for day:  1 [1, 11, 17, 28, 31, 31, 32, 34, 35, 38, 42, 53, 57, 65, 79, 102, 105, 114, 119, 120, 128, 128, 132, 133]
generating for day:  2 [6, 23, 47, 53, 68, 78, 82, 84, 91, 99, 100, 117, 121, 127, 128, 131, 140, 141, 142, 143, 148]
generating for day:  3 [0, 0, 36, 42, 43, 48, 48, 49, 69, 85, 93, 94, 95, 98, 112, 113, 114, 127, 127, 131, 138, 149]
Simulating delivery of 90 parcels over 4 days to 150 customers
[ 0] 08:33:58.6 Parcel:   0 (  9) arr at delivery centre
[ 0] 08:59:52.9 Parcel:   1 (  9) arr at delivery centre
[ 0] 09:09:50.8 Parcel:   2 ( 25) arr at delivery centre
[ 0] 09:15:19.1 Parcel:   3 ( 29) arr at delivery centre
[ 0] 09:28:23.5 Parcel:   4 ( 32) arr at delivery centre
[ 0] 09:37:52.2 Parcel:   5 ( 39) arr at delivery centre
[ 0] 10:05:50.2 Parcel:   6 ( 58) arr at delivery centre
[ 0] 10:12:26.2 Parcel:   7 ( 72) arr at delivery cen

### 20 days of verification

In [40]:
random.seed(6725)
W = generateWarehouseLocation(M)
rec, driver= simulation(M, W, C, operational_cost, p=0.15, days=20, log=True)
rec.reportTimer()


generating for day:  0 [6, 23, 47, 48, 49, 53, 68, 69, 78, 84, 85, 95, 99, 100, 117, 121, 127, 131, 140, 141, 142, 143, 148]
generating for day:  1 [0, 0, 23, 36, 42, 43, 48, 62, 82, 93, 94, 98, 98, 112, 113, 114, 121, 123, 127, 127, 130, 131, 138, 149]
generating for day:  2 [2, 10, 11, 14, 19, 32, 36, 38, 41, 42, 43, 50, 51, 58, 77, 80, 86, 104, 115, 128, 148]
generating for day:  3 [2, 3, 4, 13, 14, 19, 24, 30, 40, 61, 67, 72, 72, 86, 94, 95, 96, 107, 112, 121, 125, 139]
generating for day:  4 [3, 11, 11, 13, 16, 18, 24, 25, 31, 32, 35, 40, 41, 43, 44, 67, 70, 71, 75, 79, 83, 93, 104, 104, 109, 110, 111, 117, 118, 118, 121, 121, 125, 129, 149]
generating for day:  5 [2, 3, 9, 27, 27, 29, 31, 35, 39, 41, 44, 46, 51, 53, 61, 67, 73, 85, 97, 108, 111, 125, 131, 137, 138, 138, 148]
generating for day:  6 [0, 25, 30, 33, 34, 34, 35, 35, 36, 36, 67, 80, 88, 91, 103, 112, 114, 116, 120, 126, 129, 133, 136]
generating for day:  7 [2, 17, 29, 33, 44, 66, 69, 84, 91, 101, 111, 117, 140, 146]


### 30 days of verification

In [44]:
random.seed(6725)
W = generateWarehouseLocation(M)
rec, driver= simulation(M, W, C, operational_cost, p=0.15, days=30, log=True)
rec.reportTimer()


generating for day:  0 [0, 14, 23, 41, 42, 43, 50, 62, 82, 93, 98, 98, 112, 114, 121, 123, 127, 127, 130, 131, 138, 148, 149]
generating for day:  1 [2, 2, 3, 10, 11, 19, 24, 30, 32, 36, 38, 42, 43, 51, 58, 77, 80, 86, 94, 104, 107, 115, 125, 128]
generating for day:  2 [4, 13, 14, 16, 19, 25, 40, 61, 67, 72, 72, 75, 86, 95, 96, 109, 110, 112, 121, 139, 149]
generating for day:  3 [11, 13, 18, 24, 31, 40, 41, 44, 70, 71, 79, 83, 93, 104, 111, 117, 118, 118, 121, 121, 125, 129]
generating for day:  4 [2, 3, 3, 9, 11, 27, 27, 29, 31, 32, 34, 35, 35, 39, 41, 43, 44, 46, 51, 53, 61, 67, 67, 73, 85, 97, 104, 108, 111, 125, 131, 137, 138, 138, 148]
generating for day:  5 [0, 17, 25, 29, 30, 33, 33, 34, 35, 35, 36, 36, 67, 80, 84, 88, 91, 103, 112, 114, 116, 120, 126, 129, 133, 136, 146]
generating for day:  6 [2, 18, 28, 39, 41, 43, 44, 66, 69, 73, 83, 85, 89, 91, 92, 93, 101, 105, 111, 117, 140, 145, 149]
generating for day:  7 [16, 24, 51, 63, 87, 91, 93, 94, 106, 116, 123, 128, 136, 147]


### 40 days of verification

In [45]:
random.seed(6725)
W = generateWarehouseLocation(M)
rec, driver= simulation(M, W, C, operational_cost, p=0.15, days=40, log=True)
rec.reportTimer()


generating for day:  0 [2, 3, 4, 13, 14, 19, 24, 30, 40, 61, 67, 72, 72, 77, 86, 94, 95, 96, 107, 112, 121, 125, 139]
generating for day:  1 [11, 13, 16, 18, 25, 31, 40, 41, 70, 71, 75, 79, 83, 93, 104, 109, 110, 111, 117, 118, 121, 121, 129, 149]
generating for day:  2 [2, 3, 11, 24, 27, 29, 32, 35, 43, 44, 51, 53, 67, 67, 104, 111, 118, 125, 125, 138, 148]
generating for day:  3 [3, 9, 27, 31, 33, 34, 35, 36, 39, 41, 44, 46, 61, 73, 85, 88, 97, 108, 131, 133, 137, 138]
generating for day:  4 [0, 2, 17, 25, 29, 30, 33, 34, 35, 35, 36, 39, 44, 66, 67, 69, 80, 84, 85, 91, 91, 101, 103, 111, 112, 114, 116, 117, 120, 126, 129, 136, 140, 145, 146]
generating for day:  5 [16, 18, 24, 28, 41, 43, 51, 63, 72, 73, 83, 87, 87, 89, 91, 92, 93, 93, 94, 105, 106, 116, 123, 128, 136, 147, 149]
generating for day:  6 [0, 4, 5, 13, 14, 20, 22, 25, 28, 32, 58, 61, 64, 68, 81, 83, 116, 120, 122, 135, 136, 139, 141]
generating for day:  7 [18, 18, 26, 34, 51, 51, 52, 55, 58, 74, 87, 92, 105, 117]
genera

In [46]:
rec1 = simulation(M, W, C, operational_cost, p=0.15, days=40, log=True, timing=True)
rec1[0].reportTimer()


generating for day:  0 [2, 3, 4, 13, 14, 19, 24, 30, 40, 61, 67, 72, 72, 77, 86, 94, 95, 96, 107, 112, 121, 125, 139]
generating for day:  1 [11, 13, 16, 18, 25, 31, 40, 41, 70, 71, 75, 79, 83, 93, 104, 109, 110, 111, 117, 118, 121, 121, 129, 149]
generating for day:  2 [2, 3, 11, 24, 27, 29, 32, 35, 43, 44, 51, 53, 67, 67, 104, 111, 118, 125, 125, 138, 148]
generating for day:  3 [3, 9, 27, 31, 33, 34, 35, 36, 39, 41, 44, 46, 61, 73, 85, 88, 97, 108, 131, 133, 137, 138]
generating for day:  4 [0, 2, 17, 25, 29, 30, 33, 34, 35, 35, 36, 39, 44, 66, 67, 69, 80, 84, 85, 91, 91, 101, 103, 111, 112, 114, 116, 117, 120, 126, 129, 136, 140, 145, 146]
generating for day:  5 [16, 18, 24, 28, 41, 43, 51, 63, 72, 73, 83, 87, 87, 89, 91, 92, 93, 93, 94, 105, 106, 116, 123, 128, 136, 147, 149]
generating for day:  6 [0, 4, 5, 13, 14, 20, 22, 25, 28, 32, 58, 61, 64, 68, 81, 83, 116, 120, 122, 135, 136, 139, 141]
generating for day:  7 [18, 18, 26, 34, 51, 51, 52, 55, 58, 74, 87, 92, 105, 117]
genera

In [42]:
rec2 = simulation(M, W, C,operational_cost,  p=0.15, days=40, log=False, timing=True)
rec2[0].reportTimer()


Simulating delivery of 90 parcels over 4 days to 150 customers
==== t:   0.00s Δt:   0.00s [addTarget]
createTables:     0.24s
createLoopH:      0.24s
==== t:   0.25s Δt:   0.24s [createLoopH]
==== t:   0.25s Δt:   0.00s [addTarget]
createTables:     0.21s
createLoopH:      0.21s
==== t:   0.46s Δt:   0.21s [createLoopH]
==== t:   0.46s Δt:   0.00s [addTarget]
createTables:     0.18s
createLoopH:      0.19s
==== t:   0.65s Δt:   0.19s [createLoopH]
==== t:   0.65s Δt:   0.00s [addTarget]
createTables:     0.24s
createLoopH:      0.24s
==== t:   0.89s Δt:   0.24s [createLoopH]
==== t:   0.89s Δt:   0.00s [addTarget]
createTables:     0.20s
createLoopH:      0.20s
==== t:   1.09s Δt:   0.20s [createLoopH]
==== t:   1.10s Δt:   0.00s [addTarget]
createTables:     0.29s
createLoopH:      0.29s
==== t:   1.39s Δt:   0.29s [createLoopH]
==== t:   1.39s Δt:   0.00s [addTarget]
createTables:     0.26s
createLoopH:      0.26s
==== t:   1.65s Δt:   0.26s [createLoopH]
==== t:   1.65s Δt:   0.00s

In [47]:
rec3 = simulation(M, W, C,operational_cost, p=0.15, days=40)
rec3[0].reportTimer()

Simulating delivery of 885 parcels over 40 days to 150 customers
Driver's Working Time for the day: 10823.03 seconds
Driver's Payment for the day 90.19€
Distance traveled for the day: 1263 m
Operational cost for the day: 0.10104€ 
Driver's Working Time for the day: 12410.46 seconds
Driver's Payment for the day 103.42€
Distance traveled for the day: 1420 m
Operational cost for the day: 0.1136€ 
Driver's Working Time for the day: 10725.88 seconds
Driver's Payment for the day 89.38€
Distance traveled for the day: 1340 m
Operational cost for the day: 0.1072€ 
Driver's Working Time for the day: 10699.71 seconds
Driver's Payment for the day 89.16€
Distance traveled for the day: 755 m
Operational cost for the day: 0.0604€ 
Driver's Working Time for the day: 14027.70 seconds
Driver's Payment for the day 116.90€
Distance traveled for the day: 2243 m
Operational cost for the day: 0.17944000000000002€ 
Driver's Working Time for the day: 13494.12 seconds
Driver's Payment for the day 112.45€
Distan